In [25]:
import numpy as np
from sympy import Matrix
import copy
import math
import importlib.util
try:
	import thewalrus
except:
	print("Unable to import `thewalrus`. Using (slower) permanent backup function." )


Unable to import `thewalrus`. Using (slower) permanent backup function.


# Simulating an Interferometer.

Simulating the quantum state evolution through an interferometer is simulating boson sampling, and as such cannot be done efficiently classically.

The simplest example of boson sampling is that of simulating a beamsplitter. It's instructive to look at this example in detail as it highlights why one must move to the second quantized picture (Fock space), as the first quantised picture fails to accurately predict the correct behaviour.

![first_second_order_system_step_response](images/bs.png)


## Naive first quantised beamsplitter treatment:

Consider the case shown in the diagram here of two incident photons on a beamsplitter. In the first quantised picture we would multiply the input state by the unitary of the beamsplitter:

\begin{equation}
\begin{pmatrix} a_{out} \\ b_{out} \end{pmatrix}= \frac{1}{\sqrt{2}}\begin{pmatrix} 1 & i \\ i & 1  \end{pmatrix} \begin{pmatrix} a_{in} \\ b_{in} \end{pmatrix}
\end{equation}


For the case of $\begin{pmatrix} a_{in} \\ b_{in} \end{pmatrix} = \begin{pmatrix} 1 \\ 1 \end{pmatrix}$, this gives$\begin{pmatrix} a_{out} \\ b_{out} \end{pmatrix} =  \frac{1}{\sqrt{2}} \begin{pmatrix} 1+i \\ 1+i \end{pmatrix}$  

So the first quantised picture gives us this superposition state as an output, but fails to capture the full behaviour as it does not properly deal with interference effects. In this case, it results in the failure to describe a **HOM dip**.


## Second quantised beamsplitter treatment:
*For an intro to Fock space see pages 9-15 of https://arxiv.org/pdf/1812.10732.pdf* [1]

In the second quantised picture every mode is described by a simple harmonic oscillator each with infinite occupation levels. When working in the second quantised picture we move away from describing transforms on states and instead work out how the creation **$\hat{a}^\dagger$** and annihilation **$\hat{a}$** operators of each mode transform under the action of the given unitary.
<div>
<img src="images/bs_fock.png" width="500"/>
</div>

In the case of the general 2x2 unitary above, the creation and annihilation operators for each mode transform under the linear mapping:
\begin{equation}
\begin{bmatrix} \hat{a}_{2} \\ \hat{a}_{3} \end{bmatrix}= \frac{1}{\sqrt{2}}\begin{bmatrix} t' & r \\ r' & t  \end{bmatrix} \begin{bmatrix} \hat{a_0} \\ \hat{a_1} \end{bmatrix}
\end{equation}

Where $ \hat{a}_{2}, \hat{a}_{3}$ are the output modes. Subbing in the unitary for a 50:50 beamsplitter we can calculate how the modes transform in the case of two input photons, as in the first part.


\begin{align}
\hat{a_0}^\dagger \hat{a_1}^\dagger \ket{0_{a0}0_{a1}} \rightarrow (\hat{a_2}^\dagger+ i\hat{a_3}^\dagger)(\hat{a_3}^\dagger+ i\hat{a_2}^\dagger)\ket{0_{a2}0_{a3}} \\
=\frac{i}{2}(\hat{a_2}^\dagger\hat{a_2}^\dagger+\hat{a_3}^\dagger\hat{a_3}^\dagger)\ket{0_{a2}0_{a3}} \\
=\frac{i}{2}(\ket{2_{a2}0_{a3}}+\ket{0_{a2}2_{a3}})
\end{align}


When inputting two photons into a two port interferometer like this there are in theory 3 possible photon number preserving outcomes. $\ket{2 0}, \ket{0 2} , \ket{1 1}$. The above is telling us that in a 50:50 beamsplitter, the interference effects are such that we never see the $\ket{1 1}$ case.

## Generalising to an arbritrary interferometer:

*Follows the notation and arguments of:https://arxiv.org/pdf/quant-ph/0406127.pdf* [2]

### Notation
Ultimatley what we want to calculate is the effect of our interferometer on a given input fock state $ \ket{n_1, n_2, ...,n_N}$, and return some output quantum state. 

Let our generalised interferomter be represented by the operator $ \hat{U}$, with associated unitary matrix $ \Lambda $. Hence the fock space mode transformations are:
\begin{equation}
\hat{a} \rightarrow \Lambda^\dagger \hat{a} \hspace{3em} \hat{a}^\dagger \rightarrow \Lambda^T \hat{a}^\dagger 
\end{equation}

Here it is also relevant to introduce some more obscure notation taken verbatim from [2]: 

"Let $\Lambda[k_1, . . . , k_m|l_1, . . . , l_m]$ be the (m × m)- matrix whose matrix elements are those of the original matrix Λ with row indices $k_1, . . . , k_m$ and column indices $l_1, . . . , l_m$. For example:

\begin{equation}
\Lambda[k_1, . . . , k_m|l_1, . . . , l_m]= \begin{pmatrix} \Lambda_{k_1l_1} & \Lambda_{k_1l_2} & \Lambda_{k_1l_3} \\ \Lambda_{k_2l_1} & \Lambda_{k_2l_2} & \Lambda_{k_2l_3} \\ \Lambda_{k_3l_1} & \Lambda_{k_3l_2} & \Lambda_{k_3l_3}  \end{pmatrix}
\end{equation}

The object $\Lambda[1^{m1},2^{m2} . . . )|1^{n1},2^{n2} . . . ]$ denotes a matrix whose entries are taken from the matrix $\Lambda$ and whose row index i occurs exactly $m_i$ times and whose column index j occurs exactly $n_j$ times, for example:
\begin{equation}
\Lambda[1^{1},2^{1},3^{1} )|1^{0},2^{2},3^{1} ]= \begin{pmatrix} \Lambda_{12} & \Lambda_{12} & \Lambda_{13} \\ \Lambda_{22} & \Lambda_{22} & \Lambda_{23} \\ \Lambda_{32} & \Lambda_{32} & \Lambda_{33}  \end{pmatrix}
\end{equation}
"

### Caculating the output state

For a state with N input modes, there are N modes to transform. The creation operators being transformed are applied $n_i$ times, where $n_i$ is the occupation number of each mode. The output state obiously also depends on the relevant entires of the unitary $\Lambda$. With this in mind it is hopefully clear that the state after transformation can be written as:

\begin{equation}
\hat{U}\ket{n_1, n_2, ...,n_N} = \prod_{i=1}^N \frac{1}{\sqrt{n_i!}} \left(  \sum^N_{k_i=1} \Lambda_{ki,}i\hat{a_{ki}}^\dagger\right)
\end{equation}




With a significant amount of algebra (shown in above reference), one can show that the transition amplitude from the input state to a given output state $ \ket{m_1, m_2, ...,m_N}$ is found via:

\begin{equation}
\bra{m_1, m_2, ...,m_N}\hat{U}\ket{n_1, n_2, ...,n_N} = \left( \prod_i n_i!\right)^{-\frac{1}{\sqrt{2}}} \left( \prod_i n_i!\right)^{-\frac{1}{\sqrt{2}}} per(\Lambda[\Omega'][\Omega]) 
\end{equation}

Here we have introduced:
\begin{align}
\Omega= (1^{n_1},2^{n_2},...,N^{n_N}) \\
\Omega'=(1^{m_1},2^{m_2},...,N^{m_N} 
\end{align}

The two product coefficients within the the equation are just combinatorical factors which can be found directly from the state transition being looked at. The term $per(\Lambda[\Omega'][\Omega])$ requires slightly more peicing apart. Following from the definitions in the notation section, $\Lambda[\Omega'][\Omega]$ is a rerformulation of the matrix of the unitary operator $\Lambda$, with several replacements made so that it describes the transition from an individual term of the input vector to a specific output state being looked at. By taking the permanent of this *transition matrix*, we can calculate the RHS of the equation. For a given output state, we must take find the product of the permanent of the transition matrix corresponding to every term of the input state. The mod^2 of this then gives us the probability of this transition occcuring. This is perhaps seen more clearly in the example below.

So we have a method of calculating the transition amplitude from our input state to *a given output state $ \ket{m_1, m_2, ...,m_N}$*. In order to calculate the full output state we must evaluate the transition amplitude from our input state to every possible output state. In order to do this we enforce photon number conservation by limiting the maximum occupation of each output mode to the total number of input photons across all input modes.
  



### Coded example with explicit steps

Lets simulate a HOM dip using the above formalism. Firstly we define our input state and unitary matrix. Note the max mode occupation for each mode is 2 (sum of input photons).


In [31]:
#Unitary beamsplitter matrix
bs_mat= Matrix([[1,1j],[1j,1]])*(1/math.sqrt(2))

#Input state with one photon in each mode
input_state={(0, 0): 0j, (0, 1): 0j, (0, 2): 0j, (1, 0): 0j, (1, 1): 1, (1, 2): 0j, (2, 0): 0j, (2, 1): 0j, (2, 2): 0j}

#Empty output state.
output_state={(0, 0): 0j, (0, 1): 0j, (0, 2): 0j, (1, 0): 0j, (1, 1): 0j, (1, 2): 0j, (2, 0): 0j, (2, 1): 0j, (2, 2): 0j}

Now define a function which calculates the transition matrix for a single term in the input state vector, and a given term in the output state $ \ket{m_1, m_2, ...,m_N}$. (i.e $\ket{010} \rightarrow \ket{100})$

In [32]:
def create_transition_matrix(unitary,input_vector,output_vector, d=complex):
		""" Function to make appropriate changes to unitary so that it represents the desired transition
			from this we can then find the permanent representing the probability of this transition.
			This function must be called for every transition probability required to be calculated.
		"""
		no_photons=int(np.sum(input_vector))
		col_swapped_matrix=np.zeros(shape=(no_photons,no_photons),dtype=d)

		#If there are more or less input photons than output channels we must reshape the matrix slightly for the following to work
		#Definitely exists a more efficient way to do this

		reshaped_unitary=np.zeros(shape=(no_photons,no_photons),dtype=d)
		col_count=0
		row_count=0

		for i in range(len(input_vector)):
			for j in range(len(input_vector)):

				if (no_photons-len(input_vector))>=0:
					reshaped_unitary[i,j]=unitary[i,j]

				elif (no_photons-len(input_vector))<0:
			
					if input_vector[i]!=0 and output_vector[j]!=0:
						reshaped_unitary[row_count,col_count]=unitary[i,j]
						col_count+=1
						row_count+=1

		#Special case of matrix with only 1 photon in and out
		if len(reshaped_unitary)==1:
			return reshaped_unitary[0]


		#Make the column swaps required for the given input vector.
		col_counter=0
		for k in range(len(input_vector)):
			if input_vector[k]==0:
				continue
			else:
				for j in range(input_vector[k]):
					col_swapped_matrix[:,col_counter+j]=copy.deepcopy(reshaped_unitary[:,k])
				col_counter+=1+j


		#Make the row swaps required for a given output vector
		transition_matrix=copy.deepcopy(col_swapped_matrix)
		row_counter=0
		for p in range(len(output_vector)):
			if output_vector[p]==0:
				continue
			else:
				for r in range(output_vector[p]):
					transition_matrix[row_counter+r,:]=copy.deepcopy(col_swapped_matrix[p,:])
				row_counter+=1+r

		
		return transition_matrix

Write a function which calculates the permanent of a given matrix: https://github.com/scipy/scipy/issues/7151

In [33]:
def calculate_permanent(M):
		""" Manual permanent function for cases where thewalrus
		fails to install. As of 04/02/21 no thewalrus wheel
		for python 3.9. Slower than thewalrus, taken from:
		https://github.com/scipy/scipy/issues/7151"""
		
		n = M.shape[0]
		d = np.ones(n)
		j =  0
		s = 1
		f = np.arange(n)
		v = M.sum(axis=0)
		p = np.prod(v)

		while (j < n-1):
			v -= 2*d[j]*M[j]
			d[j] = -d[j]
			s = -s
			prod = np.prod(v)
			p += s*prod
			f[0] = 0
			f[j] = f[j+1]
			f[j+1] = j+1
			j = f[0]
		return p/2**(n-1)

Now we need to loop over every element in the input state with a non 0 amplitude and calculate every transition probability for that given element. i.e for if we have a  $\ket{11}$ input term we must evaulate the transition amplitudes to all number preserving outputs, $\ket{11},\ket{02},\ket{20}$. Each component of the output state will have transition amplitudes from multiple parts of the input state and these must be summed. For example $\ket{10}$ will have contributions from $\ket{10},\ket{01}$ input terms. Once we have done this looping over every input term, for every possible output term we are returned our output state vector.

In [34]:
def calculate_output_amplitudes(unitary, input_vector):
		"""Using the probability expression in 'Permanents in linear optical networks' Scheel 2004,
		we calculate the probability of each transition and store it in an array.
		In the fully quantum case we need to calculate all possible contributions to the output state
		that is we need to loop over every element in the input state with a non 0 amplitude
		and calculate every transition probability for that element.
		"""
		state_vector_elements=[list(key) for key in input_vector]
		input_amplitudes=list(input_vector.values() )
		output_amplitudes=np.zeros(shape=(len(input_amplitudes)), dtype=complex)

		#If the walrus not installed use manual permanent calc
		is_walrus_alive = importlib.util.find_spec(name='thewalrus')
	

	
		#For every element of the input state vector
		
		for i in range(len(state_vector_elements)):
			input_element=state_vector_elements[i]
			#Loop over every possible outcome
			for k in range(len(state_vector_elements)):
				element=state_vector_elements[k]

				#If it has a non zero amplitude
				#only consider photon number preserving transitions as these should evaluate to 0 anyway (true?)
				if input_amplitudes[i] != 0 and np.sum(input_element)==np.sum(element): 
				
					#print('The transition being calculated is ', input_element, element )

					trans_matrix=create_transition_matrix(unitary, input_element, element)


					if len(trans_matrix)==1:
						output_amplitudes[i]+=(np.abs(trans_matrix[0])**2)*input_amplitudes[i]
						
					else:
						prefactor=1

						if is_walrus_alive is None:
							perm=calculate_permanent(trans_matrix)
						else:
							perm=thewalrus.perm(trans_matrix)
						
						for m in range(len(input_element)):
							prefactor=prefactor*(1/math.sqrt(math.factorial(input_element[m])))*(1/math.sqrt(math.factorial(element[m])))
						
						output_amplitudes[k]+=np.around(perm*prefactor, decimals=6)*input_amplitudes[i]
						
		
		return output_amplitudes

Now we feed out input state of one photon in each mode into out beamsplitter and calculate the output state:

In [37]:
output_amplitudes=calculate_output_amplitudes(bs_mat, input_state)

#update the output state dictionary with the new amplitudes
it=0
for key in output_state:
		output_state[key]=output_amplitudes[it]
		it+=1

print('The output state is:', output_state)

The output state is: {(0, 0): 0j, (0, 1): 0j, (0, 2): 0.707107j, (1, 0): 0j, (1, 1): 0j, (1, 2): 0j, (2, 0): 0.707107j, (2, 1): 0j, (2, 2): 0j}


What you see above should hopefully be $\frac{i}{\sqrt(2)}(\ket{20}+\ket{02})$. We've simulated a HOM dip! 

Hopefully its clear that this is an extremely expensive calculation for large interferometers. The complexity of this calculation in general is exactly the complexity of classically simulating boson sampling (exactly what this example is). 